# Keras

__Keras Documentation__
- [Keras Code Docs](https://www.tensorflow.org/api_docs/python/tf/keras)
- [Keras Overview](https://www.tensorflow.org/guide/keras/overview)
- [Functional API](https://www.tensorflow.org/guide/keras/functional)
- [Keras Model Training](https://www.tensorflow.org/guide/keras/train_and_evaluate)
- [Custom Model Creation](https://www.tensorflow.org/guide/keras/custom_layers_and_models)
- [Custom Callbacks](https://www.tensorflow.org/guide/keras/custom_callback)
- [Mixed Precision](https://www.tensorflow.org/guide/keras/mixed_precision)
- [Keras FAQs](https://keras.io/getting-started/faq/)



__Examples__
- [RNNs w Keras](https://www.tensorflow.org/guide/keras/rnn)
- [Masking and Padding](https://www.tensorflow.org/guide/keras/masking_and_padding)
- [Paper Implementations](https://github.com/GauravBh1010tt/DeepLearn)
- [Model Architectures w Keras](https://github.com/fchollet/deep-learning-models)

In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0


## Building Networks



### Sequential Models `tf.keras.Sequential()`

In [4]:
#Pass Sequential a list of Keras layers
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(100,)),
    tf.keras.layers.Dense(30, activation='relu')])

#use the .add() method to append a new layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                310       
Total params: 13,440
Trainable params: 13,440
Non-trainable params: 0
_________________________________________________________________


### Functional API [link](https://www.tensorflow.org/guide/keras/functional)

In [5]:
inputs = tf.keras.Input(shape=100, batch_size=32)
dense1 = tf.keras.layers.Dense(100, activation='relu')(inputs)
dense2 = tf.keras.layers.Dense(30, activation='relu')(dense1)
outputs = tf.keras.layers.Dense(10, activation='softmax')(dense2)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(32, 100)]               0         
_________________________________________________________________
dense_9 (Dense)              (32, 100)                 10100     
_________________________________________________________________
dense_10 (Dense)             (32, 30)                  3030      
_________________________________________________________________
dense_11 (Dense)             (32, 10)                  310       
Total params: 13,440
Trainable params: 13,440
Non-trainable params: 0
_________________________________________________________________


### Model Subclassing

In [14]:
class MyModel(k.Model):
    def __init__(self):
        super(MyModel, self).__init__(name="Model")
        self.dense1 = k.layers.Dense(100)
        self.dense2 = k.layers.Dense(30)
        self.outputs = k.layers.Dense(10, activation='softmax')    
        
    def call(self, Inputs):
        x = self.dense1(Inputs)
        x = self.dense2(x)
        return self.outputs(x)

model = MyModel()
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
#cant call model.summarize until the model is fitted

## Model Methods

`tf.keras.Model` is the base class used by the functional and subclassing model implementations. if you subclass `Model` you can add a `training` argument to the `call` method to specify a different behavior at inference.
- `Model.layers` returns a list of the model's layers
- `.get_layer(name=__, index=__)` will reurn a layer within layers

### `model.compile()`

Configures the model for training.

`model.compile(loss=___, optimizer=___, metrics=____)`

#### Keras Losses [docs](https://www.tensorflow.org/api_docs/python/tf/keras/losses)
You instantiate a loss object and `call`ing the loss obj with `y_true` and `y_pred` will output a tensor with a scalar loss value.
- `tf.keras.losses.BinaryCrossentropy()`
- `tf.keras.losses.CategoricalCrossentropy()`
- `tf.keras.losses.Huber()`
- `tf.keras.losses.KLDivergence()`
- `tf.keras.losses.MeanAbsoluteError()`
- `tf.keras.losses.SparseCategoricalCrossentropy()`

Note that `SparseCategoricalCrossentropy` requires as input integers NOT in OHE format. ex. [1,4,45,...]

In [4]:
bce = tf.keras.losses.BinaryCrossentropy()
loss = bce([0., 0., 1., 1.], [1., 1., 1., 0.]) #outputs a result tensor
print("Loss: ", loss.numpy())

Loss:  11.522857


#### Keras Optimizers [docs](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)
Optimizers have `apply_gradients` which is passed (gradients, variable) to update the variables.

- `tf.keras.optimizers.Adadelta()`
- `tf.keras.optimizers.Adam()`
- `tf.keras.optimizers.Nadam()`
- `tf.keras.optimizers.RMSprop()`
- `tf.keras.optimizers.SGD()`

#### Keras Metrics [docs](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

__Classes__
Use the methods `update_state(y_true, y_pred)` to accumulate statisitcs. Use `.result()` to compute and return the metric tensor. `.reset_states()` to reset the metric state variables.This implementation of metrics allow output metrics to be computed across many batches

- `tf.keras.metrics.accuracy()`
- `tf.keras.metrics.BinaryCrossentropy()`
- `tf.keras.metrics.CategoricalCrossentropy()`
- `tf.keras.metrics.KLDivergence()`
- `tf.keras.metrics.MeanSquaredError()`
- `tf.keras.metrics.Precision()`
- `tf.keras.metrics.Recall()`
- `tf.keras.metrics.RootMeanSquaredError()`
- `tf.keras.metrics.AUC()`
- `tf.keras.metrics.SparseCategoricalAccuracy()`
- `tf.keras.metrics.SparseCategoricalCrossentropy()`

Note that the CrossEntropy metrics have arguments `from_logits` and `label_smoothing` 

In [3]:
m = tf.keras.metrics.Accuracy()
_ = m.update_state([1,2,3,4],[1,2,3,5])
print("First batch: ",m.result().numpy())

m.update_state([1,2,3,4],[0,0,0,0])
print("Second batch: ",m.result().numpy())

m.reset_states()
print("After Reset:",m.result().numpy())

First batch:  0.75
Second batch:  0.375
After Reset: 0.0


### `model.fit()`
`model.fit(x, y, batch_size=___, epochs=____)`


### `model.evaluate()`
`model.evaluate(x_test, y_test)`


### `model.predict()`
`model.predict(x, batch_size=None, verbose=)`

### `model.save()`


### `model.save_weights()`

*While the formats are the same, do not mix `save_weights` and `tf.train.Checkpoint`. Checkpoints saved by `Model.save_weights` should be loaded using `Model.load_weights`. Checkpoints saved using `tf.train.Checkpoint`.save should be restored using the corresponding `tf.train.Checkpoint.restore`. Prefer tf.train.Checkpoint over save_weights for training checkpoints.*

### `model.summary()`

### `model.train_on_batch()`

Good for initializing the input of a model subclassing `keras.Model` , also could be used to debug a model

## Keras Layers [docs](https://www.tensorflow.org/api_docs/python/tf/keras/layers)

`tf.keras.layers` share the same common constructor arguments 
- `activation`
- `kernel_initializer` and `bias_initializer`
- `kernel_regularizer` and `bias_regularizer`

__Notable Examples__
- `tf.keras.layers.Additiv2Attention()`[*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/AdditiveAttention) - Bahdanau Attention
- `tf.keras.layers.Attention()`[*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Attention) - Luong Attention
- `tf.keras.layers.AveragePooling2D()`
- `tf.keras.layers.BatchNormalization()`[*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization)
- `tf.keras.layers.Conv2D()`[*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)
- `tf.keras.layers.Conv2DTranspose()`[*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose)

- `tf.keras.layers.Dense()`[*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)
- `tf.keras.layers.DepthwiseConv2D`[*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/DepthwiseConv2D)
- `tf.keras.layers.Embedding()` [*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)
- `tf.keras.layers.GRU()`[*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU)
- `tf.keras.layers.Dropout()` [*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)
- `tf.keras.layers.GlobalAveragePooling2D() 
- `tf.keras.layers.GlobalMaxPool2D()`
- `tf.keras.layers.LSTM()`[*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)
- `tf.keras.layers.MaxPool2D()`
- `tf.keras.layers.ReLU()`
- `tf.keras.layers.SeparableConv2D()` [*](https://www.tensorflow.org/api_docs/python/tf/keras/layers/SeparableConv2D)
- `tf.keras.layers.Softmax()`
- `tf.keras.layers.UpSampling2D()`
- `tf.keras.layers.ZeroPadding2D()`

## Keras Callbacks [docs](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks)
- `tf.keras.callbacks.ModelCheckpoint()`
- `tf.keras.callbacks.LearningRateScheduler()`
- `tf.keras.callbacks.EarlyStopping()`
- `tf.keras.callbacks.TensorBoard()`

create a list of callbacks and pass it to the models `.fit()` method

In [ ]:
cb = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
    tf.keras.callbacks.TensorBoard(log_dir="./logs")
]
model.fit(X, y, epochs=1=, callbacks=cb)